In [95]:
#Part 1

import pandas as pd

df = pd.read_csv('/Users/iavompe/Desktop/test/Data Analyst Home Assignment/SeasonFixtures.csv')
df[['Score','Score_1hf']] = df['Score'].str.split(' ',expand=True)
df = df.drop('Score_1hf', axis=1)
df[['score_t1','score_t2']] = df['Score'].str.split(':',expand=True).astype(int)

team_list = df['HomeTeam'].unique().tolist()

teams_info = []

for team in team_list:
    games_played = df.loc[(df['HomeTeam'] == team)].shape[0] * 2
    scored_total = df.loc[(df['HomeTeam'] == team), 'score_t1'].sum() + df.loc[(df['AwayTeam'] == team), 'score_t2'].sum()
    recieved_total = df.loc[(df['HomeTeam'] == team), 'score_t2'].sum() + df.loc[(df['AwayTeam'] == team), 'score_t1'].sum()
    wins = df.loc[((df['HomeTeam'] == team) & (df['score_t1'] > df['score_t2'])), 'HomeTeam'].count() + df.loc[((df['AwayTeam'] == team) & (df['score_t1'] < df['score_t2'])), 'HomeTeam'].count()
    draws = df.loc[((df['score_t1'] == df['score_t2']) & ((df['HomeTeam'] == team) | (df['AwayTeam'] == team))), 'HomeTeam'].count()
    losses = games_played - wins - draws
    team_info = [team, games_played, wins, draws, losses, scored_total, str(str(scored_total)+':'+str(recieved_total)), scored_total - recieved_total, (wins*3+draws)]
    teams_info.append(team_info)
    

df_result = pd.DataFrame(teams_info, columns =['TeamName','GamesPlayed','Wins','Draws','Losses','Scored','Scored:Received','GoalsDifference','Points'])

df_result = df_result.sort_values(['Points', 'GoalsDifference', 'Scored'], ascending=[False, False, False])

df_result['Position'] = df_result[['Points', 'GoalsDifference', 'Scored']].apply(tuple,axis=1).rank(method='dense',ascending=False).astype(int)

df_result = df_result.loc[:, ['Position','TeamName','GamesPlayed','Wins','Draws','Losses','Scored:Received','GoalsDifference','Points']]
df_result = (df_result.sort_values('Position')).reset_index(drop=True)
df_result


,Position,TeamName,GamesPlayed,Wins,Draws,Losses,Scored:Received,GoalsDifference,Points
0,1,Manchester City,38,29,6,3,99:26,73,93
1,2,Liverpool FC,38,28,8,2,94:26,68,92
2,3,Chelsea FC,38,21,11,6,76:33,43,74
3,4,Tottenham Hotspur,38,22,5,11,69:40,29,71
4,5,Arsenal FC,38,22,3,13,61:48,13,69
5,6,Manchester United,38,16,10,12,57:57,0,58
6,7,West Ham United,38,16,8,14,60:51,9,56
7,8,Leicester City,38,14,10,14,62:59,3,52
8,9,Brighton & Hove Albion,38,12,15,11,42:44,-2,51
9,10,Wolverhampton Wanderers,38,15,6,17,38:43,-5,51


#Part 2
#SQL
create table via_tmp as 
SELECT round, hometeam, awayteam, score,
    substr(Score, 1, instr(Score, ':') - 1) AS scored_t1,
    substr(Score, instr(Score, ':') + 1) AS scored_t2
from (
    SELECT round, hometeam, awayteam, substr(Score, 1, instr(Score, ' ') - 1) AS score
    FROM SeasonFixtures
) t;


SELECT DENSE_RANK() OVER (ORDER BY wins*3+draws*1 desc, scored-recieved desc, scored desc) as Position,
    hometeam as TeamName, GamesPlayed, Wins, Draws, Losses,
    scored||':'||recieved as "Scored:Received",
    scored-recieved as GoalsDifference,
    wins*3+draws*1 as Points
from (
  SELECT hometeam, max(round) as GamesPlayed,
      sum(case when scored_t1>scored_t2 then 1 else 0 end) as Wins,
      sum(case when scored_t1=scored_t2 then 1 else 0 end) as Draws,
      sum(case when scored_t1<scored_t2 then 1 else 0 end) as Losses,
      sum(scored_t1) as scored, sum(scored_t2) as recieved
  from (
    SELECT round, hometeam, awayteam, scored_t1, scored_t2
    from via_tmp
        UNION
    SELECT round, awayteam as hometeam, hometeam as awayteam, scored_t2 as scored_t1, scored_t1 as scored_t2
    from via_tmp
  ) t
  group by 1
) t

In [166]:
#Part 3

#1
import warnings
warnings.filterwarnings("ignore")

import pandas as pd

df = pd.read_csv('/Users/iavompe/Desktop/test/Data Analyst Home Assignment/preprocessing_91.log', sep=' - ', header=None)
df.columns = ['date', 'source', 'type', 'text']

df_1 = df.copy()
df['index_col'] = df.index
df_1['index_col'] = df_1.index + 1

df_new = df.merge(df_1, how='left', left_on='index_col', right_on='index_col', suffixes=('', '_r'))

df_result = df_new.loc[(df_new['type'] == 'WARNING') & df_new['text'].str.contains('brand')==True]
df_result = df_result[['text','text_r']]

df_result[['text','text_1']] = df_result['text'].str.split(' category ',expand=True)
df_result[['status','brand']] = df_result['text_1'].str.split('brand ',expand=True)
df_result[['text','status']] = df_result['text'].str.split('due to ',expand=True)
df_result = df_result[['brand','status','text_r']].reset_index(drop=True)
df_result = df_result.rename({'text_r': 'reason'}, axis=1)
df_result

,brand,status,reason
0,Toyota,minimum series length issue,We have only 55 days of data in the last 90 days
1,Volkswagen,minimum series length issue,We have only 1 days of data in the last 90 days
2,Daimler,minimum series length issue,We have only 2 days of data in the last 90 days
3,Ford,minimum series length issue,We have only 1 days of data in the last 90 days
4,Honda,minimum series length issue,We have only 7 days of data in the last 90 days
5,BMW,minimum series length issue,We have only 1 days of data in the last 90 days
6,Hyundai,minimum series length issue,We have only 12 days of data in the last 90 days
7,Nissan,minimum series length issue,We have only 1 days of data in the last 90 days
8,Kia,minimum series length issue,We have only 6 days of data in the last 90 days
9,Renault,minimum series length issue,We have only 1 days of data in the last 90 days


#Part 3

#2

one day for Geely is not enough to predict...

1 - Determine what kind of day it is - holiday or a working day. On non-working days, sales can be significantly higher. How sales compare on different days can be understood from ready models or historical data.
2 - Determine the general trend for other brands: it grows or it is flat or it is go down.
3 - Based on the data from point 1, make a forecast for 30 days, taking into holidays and working days and global trend of sales.
4 - Using the general trend and the generated data in point 3 you can make a sales forecast

#Part 3

#3
add brand to each row
add row info: reason or status. Or make 1 row with reason and status with separator
add separator between each type of information: date, brand, reason, status, source ...

Main idea: make log more informative due to the horizontal structure and not the vertical one